In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [2]:
data = {'date':[],'team_1':[], 'team_2':[], 'score_1':[],
        'score_2':[], 'players1' : [], 'players2': [], 'Map':[],
        'team_rank_1':[], 'team_rank_2':[], 'right_pick':[], 'mean_rating1':[],
        'mean_rating2': []}
df = pd.DataFrame(data)

In [11]:
%%time
url = 'https://www.hltv.org/results?offset=8000'

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 YaBrowser/20.12.3.140 Yowser/2.5 Safari/537.36'
}

response = requests.get(url, headers = headers)
time.sleep(1)
soup = bs(response.text, 'lxml')
blocks = soup.find_all('div', class_='result-con')
past_page = ''
for k in range(1, 80):
    for i in range(len(blocks)-1,-1 ,-1):
        try:
            time.sleep(0.5)
            link = 'https://www.hltv.org' + blocks[i].find('a').get('href')
            response = requests.get(link, headers = headers)
            soup1 = bs(response.text, 'lxml')
            players_of_team1 = []
            players_of_team2 = []
            players = soup1.find_all('td', class_='player')
            for i in range(5):
                player1_html = players[i+5]
                player2_html = players[-i-1]
                player1 = player1_html.find('div', class_="text-ellipsis").text
                player2 = player2_html.find('div', class_="text-ellipsis").text
                players_of_team1.append(player1)
                players_of_team2.append(player2)

            table = soup1.find_all('table', class_='table totalstats')
            mean_rating1 = 0
            count = 0
            for i in table[0].find_all('td', class_='rating text-center')[1:]:
                count += 1
                mean_rating1 += float(i.text)
            mean_rating1 /= count

            mean_rating2 = 0
            count = 0
            for i in table[1].find_all('td', class_='rating text-center')[1:]:
                count += 1
                mean_rating2 += float(i.text)
            mean_rating2 /= count

            maps = soup1.find_all('div', class_='mapholder') #смотрит все сыграные карты одного матча
            ranks = soup1.find_all('div', class_='teamRanking') #смотрит рейтинги команд, участвующих в матче
            if ranks[1].text != '\nUnranked\n' and ranks[0].text != '\nUnranked\n': #выбрасываем матчи,где команды не имеют ранга
                for j in range(len(maps)): # идем по всем картам
                    if maps[j].find('div', class_='results-left won pick') != None: # во всех условиях выкидываем карты, которые не были отыграны
                            data_unix = soup1.find('div',class_='date').get('data-unix')
                            date_match = datetime.fromtimestamp(int(data_unix) / 1000).strftime('%Y-%m-%d')
                            teams = maps[j].find_all('div', class_='results-teamname text-ellipsis')
                            scores = maps[j].find_all('div', class_='results-team-score')
                            map = maps[j].find('div', class_='mapname').text
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[0].text, 'team_2': teams[1].text, 'score_1': scores[0].text,
                                'score_2': scores[1].text, 'players1': players_of_team1,'players2': players_of_team2,
                                'Map': map, 'team_rank_1': ranks[0].text.split('#')[1],
                                'team_rank_2': ranks[1].text.split('#')[1],
                                'right_pick': 0,
                                'mean_rating1': mean_rating1,
                                'mean_rating2': mean_rating2},
                                ignore_index=True)
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[1].text, 'team_2': teams[0].text, 'score_1': scores[1].text,
                                'score_2': scores[0].text, 'players1': players_of_team2,'players2': players_of_team1,
                                'Map': map, 'team_rank_1': ranks[1].text.split('#')[1],
                                'team_rank_2': ranks[0].text.split('#')[1],
                                'right_pick': 1,
                                'mean_rating1': mean_rating2,
                                'mean_rating2': mean_rating1},
                                ignore_index=True)

                    elif maps[j].find('div', class_='results-left lost pick') != None:
                            data_unix = soup1.find('div',class_='date').get('data-unix')
                            date_match = datetime.fromtimestamp(int(data_unix) / 1000).strftime('%Y-%m-%d')
                            teams = maps[j].find_all('div', class_='results-teamname text-ellipsis')
                            scores = maps[j].find_all('div', class_='results-team-score')
                            map = maps[j].find('div', class_='mapname').text
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[0].text, 'team_2': teams[1].text, 'score_1': scores[0].text,
                                'score_2': scores[1].text, 'players1': players_of_team1,'players2': players_of_team2,
                                'Map': map, 'team_rank_1': ranks[0].text.split('#')[1],
                                'team_rank_2': ranks[1].text.split('#')[1],
                                'right_pick': 0,
                                'mean_rating1': mean_rating1,
                                'mean_rating2': mean_rating2},
                                ignore_index=True)
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[1].text, 'team_2': teams[0].text, 'score_1': scores[1].text,
                                'score_2': scores[0].text, 'players1': players_of_team2,'players2': players_of_team1,
                                'Map': map, 'team_rank_1': ranks[1].text.split('#')[1],
                                'team_rank_2': ranks[0].text.split('#')[1],
                                'right_pick': 1,
                                'mean_rating1': mean_rating2,
                                'mean_rating2': mean_rating1},
                                ignore_index=True)



                    elif maps[j].find('div', class_='results-left won') != None:
                            data_unix = soup1.find('div',class_='date').get('data-unix')
                            date_match = datetime.fromtimestamp(int(data_unix) / 1000).strftime('%Y-%m-%d')
                            teams = maps[j].find_all('div', class_='results-teamname text-ellipsis')
                            scores = maps[j].find_all('div', class_='results-team-score')
                            map = maps[j].find('div', class_='mapname').text
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[0].text, 'team_2': teams[1].text, 'score_1': scores[0].text,
                                'score_2': scores[1].text, 'players1': players_of_team1,'players2': players_of_team2,
                                'Map': map, 'team_rank_1': ranks[0].text.split('#')[1],
                                'team_rank_2': ranks[1].text.split('#')[1],
                                'right_pick': 1,
                                'mean_rating1': mean_rating1,
                                'mean_rating2': mean_rating2},
                                ignore_index=True)
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[1].text, 'team_2': teams[0].text, 'score_1': scores[1].text,
                                'score_2': scores[0].text, 'players1': players_of_team2,'players2': players_of_team1,
                                'Map': map, 'team_rank_1': ranks[1].text.split('#')[1],
                                'team_rank_2': ranks[0].text.split('#')[1],
                                'right_pick': 0,
                                'mean_rating1': mean_rating2,
                                'mean_rating2': mean_rating1},
                                ignore_index=True)



                    elif maps[j].find('div', class_='results-left lost') != None:
                            data_unix = soup1.find('div',class_='date').get('data-unix')
                            date_match = datetime.fromtimestamp(int(data_unix) / 1000).strftime('%Y-%m-%d')
                            teams = maps[j].find_all('div', class_='results-teamname text-ellipsis')
                            scores = maps[j].find_all('div', class_='results-team-score')
                            map = maps[j].find('div', class_='mapname').text
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[0].text, 'team_2': teams[1].text, 'score_1': scores[0].text,
                                'score_2': scores[1].text, 'players1': players_of_team1,'players2': players_of_team2,
                                'Map': map, 'team_rank_1': ranks[0].text.split('#')[1],
                                'team_rank_2': ranks[1].text.split('#')[1],
                                'right_pick': 1,
                                'mean_rating1': mean_rating1,
                                'mean_rating2': mean_rating2},
                                ignore_index=True)
                            df = df.append({
                                'date': date_match,
                                'team_1': teams[1].text, 'team_2': teams[0].text, 'score_1': scores[1].text,
                                'score_2': scores[0].text, 'players1': players_of_team2,'players2': players_of_team1,
                                'Map': map, 'team_rank_1': ranks[1].text.split('#')[1],
                                'team_rank_2': ranks[0].text.split('#')[1],
                                'right_pick': 0,
                                'mean_rating1': mean_rating2,
                                'mean_rating2': mean_rating1},
                                ignore_index=True)
        except:
            pass
            
                    
    past_page = soup.find('a', class_='pagination-prev').get('href')
    link_to_past_page = 'https://www.hltv.org' + past_page
    response = requests.get(link_to_past_page, headers = headers)
    soup = bs(response.text, 'lxml')
    blocks = soup.find_all('div', class_='result-con')

TypeError: can only concatenate str (not "NoneType") to str

In [12]:
df

,date,team_1,team_2,score_1,score_2,players1,players2,Map,team_rank_1,team_rank_2,right_pick,mean_rating1,mean_rating2
0,2021-05-18,R!OT,Overperformers,2,16,"[ObZen, cheeseball, Anima, Fiend, Scr1pt]","[slipary, Oily, Mr Shark, Gregas, bandit]",Vertigo,182,173,1.0,1.114,1.010
1,2021-05-18,Overperformers,R!OT,16,2,"[slipary, Oily, Mr Shark, Gregas, bandit]","[ObZen, cheeseball, Anima, Fiend, Scr1pt]",Vertigo,173,182,0.0,1.010,1.114
2,2021-05-18,R!OT,Overperformers,16,13,"[ObZen, cheeseball, Anima, Fiend, Scr1pt]","[slipary, Oily, Mr Shark, Gregas, bandit]",Nuke,182,173,0.0,1.114,1.010
3,2021-05-18,Overperformers,R!OT,13,16,"[slipary, Oily, Mr Shark, Gregas, bandit]","[ObZen, cheeseball, Anima, Fiend, Scr1pt]",Nuke,173,182,1.0,1.010,1.114
4,2021-05-18,R!OT,Overperformers,16,11,"[ObZen, cheeseball, Anima, Fiend, Scr1pt]","[slipary, Oily, Mr Shark, Gregas, bandit]",Inferno,182,173,1.0,1.114,1.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27763,2022-06-17,Natus Vincere,FaZe,16,1,"[sdy, Perfecto, electroNic, b1t, s1mple]","[rain, broky, Twistzz, ropz, karrigan]",Inferno,3,1,1.0,1.466,0.676
27764,2022-06-17,Vitality,ENCE,16,10,"[ZywOo, misutaaa, apEX, Magisk, dupreeh]","[Spinx, Maden, Snappi, hades, dycha]",Overpass,10,2,0.0,1.164,0.906
27765,2022-06-17,ENCE,Vitality,10,16,"[Spinx, Maden, Snappi, hades, dycha]","[ZywOo, misutaaa, apEX, Magisk, dupreeh]",Overpass,2,10,1.0,0.906,1.164
27766,2022-06-17,Vitality,ENCE,19,17,"[ZywOo, misutaaa, apEX, Magisk, dupreeh]","[Spinx, Maden, Snappi, hades, dycha]",Nuke,10,2,1.0,1.164,0.906


In [13]:
df['score_1'] = df['score_1'].astype('int')
df['score_2'] = df['score_2'].astype('int')

In [14]:
df['score'] = 1 / (1 + np.exp(-(df['score_1'] / df['score_2']) + 1)**(5/4))
df.loc[df['score'].copy() < 0.5, 'score'] = 1 - 1 / \
                                                (1 + np.exp(-(df.loc[df['score'].copy() < 0.5, 'score_2'] /
                                                              df.loc[df['score'].copy() < 0.5, 'score_1'])**(5/4) + 1))

In [15]:
df['date'] = pd.to_datetime(df['date'] )

In [28]:
df['DeltaTime'] = (df['date'] - df['date'][0]).dt.days

In [18]:
columns_titles = ['date', 'team_1', 'team_2',
                  'players1', 'players2', 'team_rank_1',
                  'team_rank_2', 'score_1', 'score_2',
                  'Map', 'right_pick', 'mean_rating1', 'mean_rating2', 'score'] 
df = df.reindex(columns=columns_titles)

In [19]:
df

,date,team_1,team_2,players1,players2,team_rank_1,team_rank_2,score_1,score_2,Map,right_pick,mean_rating1,mean_rating2,score
0,2021-05-18,R!OT,Overperformers,"[ObZen, cheeseball, Anima, Fiend, Scr1pt]","[slipary, Oily, Mr Shark, Gregas, bandit]",182,173,2,16,Vertigo,1.0,1.114,1.010,0.000004
1,2021-05-18,Overperformers,R!OT,"[slipary, Oily, Mr Shark, Gregas, bandit]","[ObZen, cheeseball, Anima, Fiend, Scr1pt]",173,182,16,2,Vertigo,0.0,1.010,1.114,0.999842
2,2021-05-18,R!OT,Overperformers,"[ObZen, cheeseball, Anima, Fiend, Scr1pt]","[slipary, Oily, Mr Shark, Gregas, bandit]",182,173,16,13,Nuke,0.0,1.114,1.010,0.571619
3,2021-05-18,Overperformers,R!OT,"[slipary, Oily, Mr Shark, Gregas, bandit]","[ObZen, cheeseball, Anima, Fiend, Scr1pt]",173,182,13,16,Nuke,1.0,1.010,1.114,0.426451
4,2021-05-18,R!OT,Overperformers,"[ObZen, cheeseball, Anima, Fiend, Scr1pt]","[slipary, Oily, Mr Shark, Gregas, bandit]",182,173,16,11,Inferno,1.0,1.114,1.010,0.638344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27763,2022-06-17,Natus Vincere,FaZe,"[sdy, Perfecto, electroNic, b1t, s1mple]","[rain, broky, Twistzz, ropz, karrigan]",3,1,16,1,Inferno,1.0,1.466,0.676,1.000000
27764,2022-06-17,Vitality,ENCE,"[ZywOo, misutaaa, apEX, Magisk, dupreeh]","[Spinx, Maden, Snappi, hades, dycha]",10,2,16,10,Overpass,0.0,1.164,0.906,0.679179
27765,2022-06-17,ENCE,Vitality,"[Spinx, Maden, Snappi, hades, dycha]","[ZywOo, misutaaa, apEX, Magisk, dupreeh]",2,10,10,16,Overpass,1.0,0.906,1.164,0.310134
27766,2022-06-17,Vitality,ENCE,"[ZywOo, misutaaa, apEX, Magisk, dupreeh]","[Spinx, Maden, Snappi, hades, dycha]",10,2,19,17,Nuke,1.0,1.164,0.906,0.536699


In [25]:
for i in range(5):
    string = f'player{i+1}_team1'
    df[string] = df['players1'].str[i]

In [26]:
for i in range(5):
    string = f'player{i+1}_team2'
    df[string] = df['players2'].str[i]

In [27]:
df = df.drop(['players1', 'players2'], axis=1)


In [43]:
columns_titles = ['date', 'DeltaTime', 'team_1', 'team_2', 'player1_team1',
                  'player2_team1', 'player3_team1', 'player4_team1', 'player5_team1',
                  'player1_team2', 'player2_team2', 'player3_team2', 'player4_team2',
                  'player5_team2', 'team_rank_1', 'team_rank_2', 'score_1', 'score_2',
                  'Map', 'score'] 
df = df.reindex(columns=columns_titles)

In [44]:
df

,date,DeltaTime,team_1,team_2,player1_team1,player2_team1,player3_team1,player4_team1,player5_team1,player1_team2,player2_team2,player3_team2,player4_team2,player5_team2,team_rank_1,team_rank_2,score_1,score_2,Map,score
0,2021-05-18,0,R!OT,Overperformers,ObZen,cheeseball,Anima,Fiend,Scr1pt,slipary,Oily,Mr Shark,Gregas,bandit,182,173,2,16,Vertigo,0.000004
1,2021-05-18,0,Overperformers,R!OT,slipary,Oily,Mr Shark,Gregas,bandit,ObZen,cheeseball,Anima,Fiend,Scr1pt,173,182,16,2,Vertigo,0.999842
2,2021-05-18,0,R!OT,Overperformers,ObZen,cheeseball,Anima,Fiend,Scr1pt,slipary,Oily,Mr Shark,Gregas,bandit,182,173,16,13,Nuke,0.571619
3,2021-05-18,0,Overperformers,R!OT,slipary,Oily,Mr Shark,Gregas,bandit,ObZen,cheeseball,Anima,Fiend,Scr1pt,173,182,13,16,Nuke,0.426451
4,2021-05-18,0,R!OT,Overperformers,ObZen,cheeseball,Anima,Fiend,Scr1pt,slipary,Oily,Mr Shark,Gregas,bandit,182,173,16,11,Inferno,0.638344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27763,2022-06-17,395,Natus Vincere,FaZe,sdy,Perfecto,electroNic,b1t,s1mple,rain,broky,Twistzz,ropz,karrigan,3,1,16,1,Inferno,1.0
27764,2022-06-17,395,Vitality,ENCE,ZywOo,misutaaa,apEX,Magisk,dupreeh,Spinx,Maden,Snappi,hades,dycha,10,2,16,10,Overpass,0.679179
27765,2022-06-17,395,ENCE,Vitality,Spinx,Maden,Snappi,hades,dycha,ZywOo,misutaaa,apEX,Magisk,dupreeh,2,10,10,16,Overpass,0.310134
27766,2022-06-17,395,Vitality,ENCE,ZywOo,misutaaa,apEX,Magisk,dupreeh,Spinx,Maden,Snappi,hades,dycha,10,2,19,17,Nuke,0.536699


In [45]:
df = df.convert_dtypes()

In [46]:
df.to_csv(r'mathes_csgo.csv', index=False)